In [1]:
import pandas as pd
import nltk

In [254]:
# %load cmdthesaurus.py
from nltk.corpus import wordnet as wn

#ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
class CmdThesaurus:

    def __init__(self):

        # verbs
        self.quitSynonyms = []
        self.openSynonyms = []
        self.showSynonyms = []
        self.calculateSynonyms = []
        self.setSynonyms = []
        # sums - total, sum, sigma
        # calculate, get
        # while, repeat

        # nouns
        self.colrows = ['column', 'columns', 'col', 'cols', 'row', 'rows']
        self.columns = ['column', 'columns', 'col', 'cols']
        self.rows = ['row', 'rows']

        # build quit synonyms
        self.quitSynonyms = buildSynList(self.quitSynonyms, "exit", wn.VERB)
        self.quitSynonyms = buildSynList(self.quitSynonyms, "quit", wn.VERB)
        self.quitSynonyms = list(set(self.quitSynonyms))

        #build read/open synonyms
        self.openSynonyms = buildSynList(self.openSynonyms, "read", wn.VERB)
        self.openSynonyms = buildSynList(self.openSynonyms, "open", wn.VERB)
        self.openSynonyms = list(set(self.openSynonyms))
        self.openSynonyms.remove('show')

        #build show/print synonyms
        self.showSynonyms = buildSynList(self.showSynonyms, "show", wn.VERB)
        self.showSynonyms = buildSynList(self.showSynonyms, "print", wn.VERB)
        self.showSynonyms = list(set(self.showSynonyms))

        #build show/print synonyms
        self.calculateSynonyms = buildSynList(self.calculateSynonyms, "get", wn.VERB)
        self.calculateSynonyms = buildSynList(self.calculateSynonyms, "calculate", wn.VERB)
        self.calculateSynonyms = buildSynList(self.calculateSynonyms, "see", wn.VERB)
        self.calculateSynonyms = buildSynList(self.calculateSynonyms, "evaluate", wn.VERB)
        self.calculateSynonyms.append("what")   # in statistical NLP, has the same purpose as calculate
        self.calculateSynonyms.append("eval")
        self.calculateSynonyms = list(set(self.calculateSynonyms))

        #build set synonyms -- need to consider set where there is no verb
        self.setSynonyms = buildSynList(self.setSynonyms, "set", wn.VERB)
        self.setSynonyms = buildSynList(self.setSynonyms, "change", wn.VERB)
        self.setSynonyms = buildSynList(self.setSynonyms, "modify", wn.VERB)
        self.setSynonyms = list(set(self.setSynonyms))

    def isColrow(self, word):
        return (word.lower() in self.colrows)

    # returns true if word is a synonym of quit
    def isQuitSynonym(self, word):
        return (word.lower() in self.quitSynonyms)

    # returns true if word is a synonym of open
    def isOpenSynonym(self, word):
        return (word.lower() in self.openSynonyms)

    # returns true if word is a synonym of show
    def isShowSynonym(self, word):
        return (word.lower() in self.showSynonyms)

    # returns true if word is a synonym of calculate
    def isCalculateSynonym(self, word):
        return (word.lower() in self.calculateSynonyms)

    # returns true if word is a synonym of set
    def isSetSynonym(self, word):
        return (word.lower() in self.setSynonyms)

# adds synonyms of words to a list
# li - the list to add to -- will return this
# word - the word to find synonyms for
# pos - the type of word it is, i.e. noun, verb, etc. as a char, see 
# the wordnet pos list, see comment at top of class
def buildSynList(li, word, pos):

    # filters out words based on pos
    for syn in [x for x in wn.synsets(word) if x.pos() == pos]:
        for l in syn.lemmas():
            li.append(l.name())
    return li


In [256]:
# %load '../Statistics/statsop.py'
import pandas as pd
import os
import string
import os.path
import fnmatch

class StatsOp:
   
    #constructor
    def __init__(self):
        self.isInitialized = False
        self.columns = []
        self.rows = []
        
#<--------------- Table Operations --------------->
    
    #update value passing in two labels Column = Header Row Name, Row = either name or index (0 - n)
    def updateCellLabel(self, col, row, value):
        if self.isInitialized:
            df = self.data
            df.set_value(row, col, value)
            self.data = df
        else:
            return None
        
    #Passing in Excell Format or updating (ex: update A4, 5)    
    def updateCellExcell(self, cell, value):
        row = ""
        col = ""
        for term in cell:
            if term.isnumeric():
                 row = row + term
            else:
                col = col + term
        rowIndex = int(row)
        num = 0
        for c in col:
            if c in string.ascii_letters:
                num = num * 26 + (ord(c.upper()) - ord('A')) + 1
        colIndex = num
        if self.isInitialized:
            df = self.data
            if (colIndex - 1) < len(list(df)):
                colName = list(df)[colIndex - 1]
                df.set_value(rowIndex, colName, value)
                self.data = df
            else:
                return None
        else:
            return None
        
    def getCellExcell(self, cell):
        row = ""
        col = ""
        for term in cell:
            if term.isnumeric():
                 row = row + term
            else:
                col = col + term
        rowIndex = int(row)
        num = 0
        for c in col:
            if c in string.ascii_letters:
                num = num * 26 + (ord(c.upper()) - ord('A')) + 1
        colIndex = num
        if self.isInitialized:
            df = self.data
            if (colIndex - 1) < len(list(df)):
                colName = list(df)[colIndex - 1]
                print(df.loc[rowIndex, colName])
                return df.loc[rowIndex, colName]
            else:
                return None
        else:
            return None
    
    #append value (row) into given column
    def insertRow(self, col, value):
        if self.isInitialized:
            df = self.data
            dfTemp = pd.DataFrame(df.iloc[[0]])
            for i, row in dfTemp.iterrows():
                for colName in list(df):
                    dfTemp.loc[i, colName] = np.nan
            dfTemp.set_value(0, col, value)
            df = df.append(dfTemp)
            df = df.reset_index(drop=True)
            self.data = df
        else:
            return None
    
#<--------------- Getter & Setter Operations --------------->
    #getter and setter for operation
    def setOperation(self, op):
        self.operation = op
        
    def getOperation(self):
        return self.operation
 
    
    #getter & setter for the filename
    def setFilename(self, fName):
        self.fileName = fName
    
    def getFilename(self):
        return self.fileName
    
    #reads in csv file into a dataframe and stores that df as data
    #TODO: handle cases where the file is not in the format of csv...
    def setData(self, fName):
        #name.csv
        hasExtension = fName.find(".")
        status = False
        if hasExtension < 0:
            filename = fName + ".csv"
            for file in os.listdir('.'):
                if fnmatch.fnmatch(file, '*.csv'):
                    if filename == file:
                        fName = filename
                        status = True
        else:
            status = os.path.isfile(fName)

        if status:
            self.data = pd.read_csv(fName)
            self.isInitialized = True
        else:
            print("404 File:" + fName +" Not Found")
        self.isInitialized = status
        return status
    
    
    def getData(self):
        if self.isInitialized:
            return self.data
        else:
            return None
        
        
#<--------------- Querying Operations --------------->
       
    def checkInitialized(self):
        return self.isInitialized
    
    #returns the mean for the given column
    def calculateColumnMean(self,col):
        df = self.data
        mean = df[col].mean()
        return mean
    
    #returns the mean for the given row
    def calculateRowMean(self, row):
        df = self.data
        mean = df.iloc[row].mean()
        return mean
    
    #returns an array of the mean for each column
    def calculateColumnsMean(self):
        if self.isInitialized:
            dataframe = self.data
            columns = self.columns
            result = []
            for col in columns:
                mean = dataframe[col].mean()
                result.append(mean)
                print(col + " " + str(mean))
            return result
        else:
            return None
       
    def describeColumn(self):
        if self.isInitialized:
            dataframe = self.data
            columns = self.columns
            result = []
            for col in columns:
                #description is a dataframe
                description = dataframe[col].describe()
                result.append(description)
            return result
        else:
            return None
    
     #returns an array of the mean for each row
    def calculateRowsMean(self):
        if self.isInitialized:
            dataframe = self.data
            rows = self.rows
            result = []
            for row in rows:
                mean = dataframe.iloc[row].mean()
                result.append(mean)
            return result
        else:
            return None

        
#<--------------- Print Operations --------------->

    #print column
    def printColumn(self,col):
        if self.isInitialized:
            df = self.data
            if(str(col).isnumeric()):
                print(df.iloc[:,:col])
            else:
                print(df[col])
        else:
            print("No Data Available")
    
    
    #print row
    def printRow(self,row):
        if self.isInitialized:
            df = self.data
            print(row)
            print(df.iloc[[row]])
        else:
            print("No Data Available")
        
    #List of all column and row names
    def getColumnNames(self):
        if self.isInitialized:
            df = self.data
            return list(df)
        else:
            return None
    
    def getRowNames(self):
        if self.isInitialized:
            df = self.data
            return list(df.index)
        else:
            return None

    def getColumn(self,col):
        if self.isInitialized:
            df = self.data
            if(str(col).isnumeric()):
                return (df.iloc[:,:col])
            else:
                return (df[col])
        else:
            return None
    
    #print row
    def getRow(self,row):
        if self.isInitialized:
            df = self.data
            return df.iloc[[row]]
        else:
            return None
        
    def isAColumn(self, col, num):
        df = self.data
        if col in list(df):
            return col
        if num > 0 and num < len(list(df)):
            return num
        return False
                    
    def isARow(self,row, num):
        df = self.data
        if row in list(df.index):
            return row
        print(len(list(df.index)))
        if num >= 0 and num < len(list(df.index)):
            return num
        return False
        
         
    
#<--------------- Array Operations --------------->

    #Unused for now...
    
    #getter & setter for columns
    #set column array = array
    def setColumns(self, cols):
        self.columns = cols
        
    #add columns to the array
    def addColumn(self, col):
        self.columns.append(col)
    
    def getColumns(self):
        return self.columns
    
    #getter & setter for row
    #set array = array
    def setRows(self, rows):
        self.rows = rows
    #add individual rows    
    def addRow(self, row):
        self.rows.append(row)
        
    def getRows(self):
        return self.rows
    
#<--------------- Misc Operations --------------->
   
    #example function    
    def testFunc(self):
        return 'hello world'
    

In [364]:
# %load synthesis.py
from nltk import word_tokenize
from nltk import tag
from enum import Enum
import re
import sys
import string

import trainer
# sys.path.insert(0, '../Statistics')
# from statsop import StatsOp
# from cmdthesaurus import CmdThesaurus

class Open(Enum):
    NOT_OPEN_CMD = 0
    OPEN_FAIL = 1
    OPEN_SUCCESS = 2

# special predecessor when parsing
class SpecialPred(Enum):
    COLUMN_SPEC = 0
    ROW_SPEC = 1 
    ORDINAL_NUMBER = 2
    APPLY_PRED = 3
    COMMA_PRED = 4
    AND_PRED = 5
    
class Node:
    def __init__(self):
        self.children = []
        self.data = ""
    
    def getChildren(self):
        return self.children
    
    def addChild(self, child):
        self.children.append(child)
    
    def getData(self):
        return self.data
    
    def setData(self, value):
        self.data = value
    
    def printData(self):
        print(self.data)

class Synthesizer:

    def __init__(self):
        self.stats = StatsOp()
        self.thesaurus = CmdThesaurus()
        self.unitagger = trainer.load_tagger('models/brown_all_uni.pkl')
        #self.bitagger = trainer.load_tagger('models/brown_all_bi.pkl')
        #self.tritagger = trainer.load_tagger('models/brown_all_tri.pkl')

        # for matching if something is a cell
        self.cellReg = re.compile('[a-zA-Z]+[0-9]+')

        self.labels = {}
        self.labels['verb'] = 'VB'
        self.labels['noun'] = 'NN'
        self.labels['nouns'] = 'NNS'

        self.applyOps = ['by', 'to', 'from']
        
        self.variables = [] # list of variables that we know
        self.commandStack = []

    def tokenize(self, cmd):
        return word_tokenize(cmd)

    def tag(self, tokens):
        return self.unitagger.tag(tokens)

    def synonym_look_up(self,word):
        #check if word is in synonynm
        #return the appropriatly mapped word
        return word
    
    def print_requested (self, objs):
        word = ""
        number = -1
        isColumn = False
        isRow = False
        for item in objs:
            if word.lower() == "column":
                isColumn = True
            elif word.lower() == "row":
                isRow = True
        
            if item.isnumeric():
                number = int(item)
                
            if len(word) > 0:
                word = word + " " + str(item)
            else:
                word = str(item)
                
        if isColumn:
            corCol = self.stats.isAColumn(word, number)
            self.stats.printColumn(corCol)
        elif isRow:
            print("processing row...: number is: " + str(number))
            corRow = self.stats.isARow(word, number)
            self.stats.printRow(corRow)
    
    # If it's an open cmd
    def read_data_cmd(self, tokens): 
        if self.thesaurus.isOpenSynonym(tokens[0][0]): #handles case of reading in data
            # next argument must be filename, attempt to set up the data
            if self.stats.setData(tokens[-1][0]):
                print(tokens[-1][0] + ' has been successfully opened')
                return Open.OPEN_SUCCESS
            else:

                print('We couldn\'t find your file: ' + tokens[-1][0])
                return Open.OPEN_FAIL
        return Open.NOT_OPEN_CMD
    
    def print_data_cmd(self, tokens):
        print ("attempting to print..")
        command = self.build_command(tokens)
        print(self.commandStack)
        if tokens[0][0].lower() == 'show': # handles cases of printing or showing data
            [self.print_requested(n) for n in self.commandStack]
            self.commandStack = []
            
    def run_data_cmd(self, tokens):
        if tokens[0][0].lower() == 'command': # handles cases statistic commands on data
            print("printing out requested data")
            

    # parse a noun or none tag and see what it is
    def check_var_or_const(self, nn):
        val = None

        # test if int
        try:
            val = int(nn)
            return val
        except ValueError:
            pass

        # test if float
        try:
            val = float(nn)
            return val
        except ValueError:
            pass

        # we now know val is a variable
        if val is None: # check if nn is even a column or row name
            return nn

    # checks if the cell passed in refers to a cell, i.e. A1, bc23
    def check_cell(self, cell):
        return self.cellReg.fullmatch(cell) != None

    # build a command stack given list of tagged tokens
    '''
    def build_command(self, tokens):
        command = []
        previousNoun = False
        for pair in tokens:
            if(pair[1] == "VB"): #action to perform
                command.append(pair[0])
                self.commandStack.append(command)
                command = []
                
            if(pair[1] == "CD" and previousNoun): #cardinal number (or location) after a given noun
                command.append(pair[0])

            if(pair[1][0] == "N"): #noun
                command.append(pair[0])
                previousNoun = True
            else:
                previousNoun = False
            
            if(pair[1] == "CC"):
                self.commandStack.append(command)
                command = []
                
        self.commandStack.append(command)
    '''
#----------------------------------------------------------------------->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    def populateTree(self, node, taggedData):
        if len(taggedData) == 0: return node
        # print("Verb? " + taggedData[0][0])
        tag = taggedData[0][1]
        while tag is None or tag[0:2] != "VB":
            newNode = Node()
            if tag is None or tag[0] == 'N' or  tag[0:2] == "CD":
#                 print("Adding Child: " + taggedData[0][0] )
                newNode.setData(taggedData[0][0])
                node.addChild(newNode)
            taggedData.pop(0)
            if len(taggedData) == 0 : return node
            tag = taggedData[0][1]


        if tag is not None and tag[0:2] == "VB":
#             print("Adding Verb and its children: " + taggedData[0][0])
            newNode = Node()
            newNode.setData(taggedData[0][0])
            taggedData.pop(0)
            node.addChild(self.populateTree(newNode, taggedData))
            return node


    def printTree(self, node, i, mylist):
        if len(node.getChildren()) == 0:
            if len(mylist) > 0:
                return (node.getData() ,mylist)
            else:
                mylist.append(node.getData())
                return ("evaluate", mylist)
            
        childLen = len(node.getChildren())
        children = node.getChildren()
        index = 0
        while index < childLen:
            if len(children[index].getChildren()) > 0:
                mylist.append(self.printTree(children[index],i + 1,[]))
            else:
                #check if the sequence is column the # 
                evals = None
                if self.thesaurus.isColrow(children[index].getData()) and (index + 1 < childLen):
                    columnList = []
                    columnList.append(children[index].getData())
                    columnList.append(children[index + 1].getData())
                    evals = ("evaluate", columnList)
                    index = index + 1  
                else:    
                    evals = ("evaluate",children[index].getData())
                mylist.append(evals)
            index = index + 1    

        if len(mylist) > 0:
            return (node.getData() ,mylist)
        else:
            mylist.append(node.getData())
            return ("evaluate", mylist)

    def generateCommand(self, tagged):
        tree = Node()
        popTree = self.populateTree(tree,tagged)
        obj = None
        if len(popTree.getChildren()) > 1:
            popTree.setData("evaluate")
            obj = self.printTree(popTree, 0,[])
        else:
            actualTree = popTree.getChildren()[0]
            obj = self.printTree(actualTree, 0,[])
       
        return obj


    # where cmd is a pair ('cmd name', [list of args])
    def execute_command(self, cmd):

        if cmd[0] is None:
            print("Something went wrong...")
            return

        # calculate command
        if self.thesaurus.isCalculateSynonym(cmd[0]):
            return self.calculate_command(cmd[1])

        # show command
        elif self.thesaurus.isShowSynonym(cmd[0]):
            return self.show_command(cmd[1])

        # set command
        elif self.thesaurus.isSetSynonym(cmd[0]):
            return self.set_command(cmd[1])



    # calculate only cares about the first argument, unless it's 
    # a special var like column 2
    def calculate_command(self, args):
        # something went wrong
        if len(args) == 0:
            print('something went wrong in a calculate command: no args')
            return

        arg = args[0]

        # if this is even a pair, we need to keep parsing
        if isinstance(arg, tuple):
            return self.execute_command(arg)
        elif isinstance(arg, str):
            # check if this is a column or row 'column __'
            if self.thesaurus.isColrow(arg):
                # if row/col number or name
                if self.stats.checkInitialized():
                    if len(args) > 1:
                        val = self.check_var_or_const(args[1])
                        if arg in self.thesaurus.columns:
                            #TODO get column
                            return self.stats.getColumn(val)
                        elif arg in self.thesaurus.rows:
                            return self.stats.getRow(val)
                        else:
                            return None
                    else: # argument list ended with col/row, haven't handled this case
                        return None
                else:
                    print('file hasn\'t been loaded so I don\'t have columns or rows yet')

            # check if this is a cell
            if self.check_cell(arg):
                if self.stats.checkInitialized():
                    return self.stats.getCellExcell(arg)
                else:
                    print('file hasn\'t been loaded so I don\'t know about ' + str(arg))

            # check if this is a variable name or constant
            val = self.check_var_or_const(arg)
            if val is None: # variable name -- must be a column
                if self.stats.checkInitialized():
                    columns = self.stats.getColumnNames()
                    if val in columns:
                        return self.stats.getColumn(val)
                    else:
                        print(val + ' is not a proper column name')
                        return None
                else:
                    print('file hasn\'t been loaded so I don\'t know about ' + str(val))
            else: # int or float
                return val


        else:
            print('Something went wrong in a calculate command: invalid arg')

    def show_command(self, args):
        # something went wrong
        if len(args) == 0:
            print('Something went wrong in a show command: no args')
            return

        arg = args[0]

        # if this is even a pair, we need to keep parsing
        if isinstance(arg, tuple):
            res = self.execute_command(arg)
            print(str(res))
            return res
        else:
            print(str(arg))
            return arg

    def set_command(self, args):
        # something went wrong
        if len(args) < 2:
            print('Something went wrong in a set command: set needs at least 2 arguments')
            return

        des = args[0]   # the destination of what we're setting
        src = args[1]   # the source of what we're setting

        if isinstance(des, tuple) and isinstance(src,tuple):
            if des[0] == 'evaluate':

                # evaluate the src value
                res = self.execute_command(src)
                if res is None:
                    print('Something went wrong when trying to calculate the value to set')
                    return

                cell = des[1][0]
                if self.check_cell(cell):
                    if self.stats.checkInitialized():
                        self.stats.updateCellExcell(cell, res)
                        return res    
                    else:
                        print('file hasn\'t been loaded so I don\'t know about ' + str(des[1][0]))

            else:
                print("Something went wrong in trying to set a value to " + str(des[1]))
        else:
            print("Something went wrong in set command: invalid arg")

    def synthesize(self, tagged, cmd):
        stats = self.stats
        labels = self.labels

        # Check if the command is an open command
        if self.read_data_cmd(tagged) != Open.NOT_OPEN_CMD:
            return # we attempted to open a file, so this command is finished

        # If we come across any nouns or nones, we need to check if it's been initialized
        # build_command call here, read is a special command
        print("building command: ")
        c = self.generateCommand(tagged)
        print('c is ' + str(c))
        res = self.execute_command(c)
        print(res)
        return res


        # Call the command -- based on the command call a list of stat ops

        # check if we've already initialized
        '''
        if stats.checkInitialized():

            # Check if cmd is a row or column
            cols = stats.getColumnNames()
            rows = stats.getRowNames()

            if check_name_in_list(cmd, cols):
                print('name found in cols')
            elif check_name_in_list(cmd, rows):
                print('name found in rows')
        

        # check if command 
        if tagged[0][1] == labels['verb']:
            print("Processing Verb")
            # Try to interpret this as a read initialization command
            if not stats.checkInitialized():
                self.read_data_cmd(tagged)
            else:
                print("Trying a non read command")
                command = self.synonym_look_up(tagged[0][0].lower())
                if command == "show":
                    self.print_data_cmd(tagged)
                pass # test printing column/row commands here?

        if tagged[0][1] == labels['noun'] or tagged[0][1] == labels['nouns']:

            print("Processing Noun")
            if not stats.checkInitialized():
                pass
            else:
                pass
        '''


In [365]:
# initialize synthesizer
s = Synthesizer()

In [366]:
# Read commands
#Set A1 to the add of 1 and 2 and 3
while(1):
    print('Enter command:')
    cmd = input()
    tokens = s.tokenize(cmd)
    tagged = s.tag(tokens)

    print(tagged)
    s.synthesize(tagged, cmd)
    if(cmd == "quit"):
        break

Enter command:
read dummydata
[('read', 'VB'), ('dummydata', None)]
dummydata has been successfully opened
Enter command:
33
[('33', 'CD')]
building command: 
Tree root data: 
c is ('evaluate', ['33'])
33
Enter command:
column 1
[('column', 'NN'), ('1', 'CD')]
building command: 
Tree root data: 
No kids
c is ('evaluate', [('evaluate', ['column', '1'])])
    Column 1
0         94
1         67
2         56
3         94
4        148
5        142
6        108
7         32
8         14
9        119
10        47
11       180
12        40
13       200
14        60
15        73
16       106
Enter command:
Set A1 to the add of 1 and 2 and 3
[('Set', 'VB'), ('A1', None), ('to', 'TO'), ('the', 'AT'), ('add', 'VB'), ('of', 'IN'), ('1', 'CD'), ('and', 'CC'), ('2', 'CD'), ('and', 'CC'), ('3', 'CD')]
building command: 
Tree root data: 
No kids
No kids
No kids
No kids
c is ('Set', [('evaluate', 'A1'), ('add', [('evaluate', '1'), ('evaluate', '2'), ('evaluate', '3')])])
Something went wrong when trying